### connect react to the supabase and CRUD

For the shake of learning, we will build a Task manager 

In [ ]:
npm install @supabase/supabase-js

supabase-client.js

In [ ]:
import {createClient} from "@supabase/supabase-js"
require('dotenv').config();


const supabaseUrl = 'https://wtjjtwictrguauiknlmt.supabase.co'
const supabaseKey = process.env.SUPABASE_KEY
export const supabase = createClient(supabaseUrl, supabaseKey)

In the app.js

In [ ]:
import "./App.css";

import { useState } from "react";
import { supabase } from "./supabase-client";

function App() {
    const [newTask, setNewTask] = useState({ task: "", description: "" });
    const handleSubmit = async (e) => {
        e.preventDefault();
        const { data, error } = await supabase.from("tasks").insert([newTask]);
        # from represent table name which have feature like insert, select, update, delete
        # here [newTask] is numbers of objects insering in bulk.
        # .multiple(True) is used to fetch multiple objects at once     
        const {data, error} = await supabase.from("tasks").insert(newTask).single();
        # newTask is a single object, since we are inserting one object at a time
        # .single() is used when we are inserting a single object
        if (error){
        console.log("Error inserting data:", error);
        return;
        }
        setNewTask({title: "",description:""})
        # resetting the title and description to null after submit
  };  

  return (
      <form onSubmit={handleSubmit} style={{ marginBottom: "1rem" }}>
        <input
          type="text"
          placeholder="Task Title"
          onChange={(e) =>
            setNewTask((prev) => ({ ...prev, task: e.target.value }))
          }
          # FIX 4: The value prop must match the state property "task".
          value={newTask.task}
          style={{ width: "100%", marginBottom: "0.5rem", padding: "0.5rem" }}
        />
        <textarea
          placeholder="Task Description"
          onChange={(e) =>
            setNewTask((prev) => ({ ...prev, description: e.target.value }))
          }
          # FIX 5: Textarea also needs a value prop to be a controlled component.
          value={newTask.description}
          style={{ width: "100%", marginBottom: "0.5rem", padding: "0.5rem" }}
        />
        # The button's type is "submit", so it automatically triggers the form's onSubmit event 
        <button type="submit" style={{ padding: "0.5rem 1rem" }}>
          Add Task
        </button>
      </form>
  )
}

Similarly, we want to fetch task as well right. let's do it

In [ ]:


function App() {
  const [tasks, setTasks] = useState([]);
  const fetchTasks = async () => {
    const { data, error } = await supabase
        .from("tasks")
        .select("*")
        .order("created_at", { ascending: true });
    
    if (error) {
      console.error("Error fetching data:", error);
      return;
    } else {
      console.log("Fetched tasks:", data);
    }
    setTasks(data);
  };
  useEffect(() => {
    fetchTasks();
    console.log(tasks);
  }, []);

  return(
      
      <ul>
        {tasks.map((task,key) => (
        <li key={key}>
          <div>
            <h3>{task.task}</h3>
            <p>{task.description}</p>
            <div>
              <button>Edit</button>
              <button>Delete</button>
            </div>
          </div>
        </li>
        ))}
      </ul>

  )
  
}

Now,we want to delete the task

In [ ]:
function app(){

    const deleteTask = async (id) => {
        const { data, error } = await supabase
            .from("tasks")
            .delete()
            .eq("id", id);
            // equivalance condition
        if (error) {
            console.error("Error deleting task:", error);
            return;
        } else {
            console.log("Task deleted successfully:", data);
            // Refresh the task list after deletion
            fetchTasks();
        }
    };

    # from above
    <button onClick={() => deleteTask(task.id)}>Delete</button>

}

Now, we need to update it too

In [ ]:

function App() {
    const [newDescription, setNewDescription] = useState(""); 

    const updateTask = async (id) => { 
        const { data, error } = await supabase
            .from("tasks")
            .update({ description: newDescription })
            .eq("id", id)
        
        if (error) {
            console.error("Error updating task:", error);
            return;
        } else {
            console.log("Task updated successfully:", data);
            // Refresh the task list after update
            fetchTasks();
        }
  };


    return(
        <textarea
            placeholder="Update Task Title"
            onChange={(e) => {setNewDescription(e.target.value )}}/>
        <button 
            style={{ padding: "0.5rem 1rem", marginRight: "0.5rem" }}
            onClick={() => updateTask(task.id, { task: newDescription })}
            >
            Edit
        </button>
    )
}